In [50]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API

import time
from tweepy import Stream
from dateutil import parser
import json

# Jack Nugent Keys
C_KEY = 'qDLzDR1kgKZrwc0nfCiknCNak'
C_SECRET = '0siNqGf4CSQVM2EL2JTYKq0fqBN7VXtTXv2MSyUAiiX5V1DBeG'
A_TOKEN_KEY = '762757536827121664-YNdvEO2FsulfLcjUJcezb5OMRx3Nlgs'
A_TOKEN_SECRET = '88DYUdSwCfWmNlBWbY9It8uFqajxE7NMMRS4moZbdjz4k'
BEARER = "AAAAAAAAAAAAAAAAAAAAAOCnhQEAAAAABJxPmDWLGJXfFdBa9Nprs%2BfFHus%3De6Qt9gjlzObpDy4HIWJWMQUXXLE6nkNUF3Drkv6QLkZezBn4JU"

In [68]:
class MyStreamingClient(tweepy.StreamingClient):
    def __init__(self, bearer_token, time_limit=240):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open('output_geotag.json', 'w')
        super(MyStreamingClient, self).__init__(bearer_token)
        
    def on_data(self, data_item):
        
        ### Modify on_data
        dict_data = {}
        try:
            data = json.loads(data_item)
            if data['coordinates'] or data['geo']:
                print("COORDS:",data['coordinates'], '\nGEO:', data['geo'])
                dict_data['created_at'] = parser.parse(data['created_at'], ignoretz=True)
                dict_data['text'] = data['text']
                dict_data['id_str'] = data['id_str']
                dict_data['user_id'] = data['user']['id_str']
                dict_data['user_name'] = data['user']['id_str']
                dict_data['lat'] = str(data['coordinates']['coordinates'][1])
                dict_data['lon'] = str(data['coordinates']['coordinates'][0])
                dict_data['language'] = data['user']['lang']
                
            print(dict_data)
            self.outFile.write(json.dumps(dict_data))
            self.outFile.write('\n')
        except:
#             print('FAIL')
            pass
        ###

        if (time.time() - self.start_time) < self.limit:
            return True
        else:
            self.outFile.close()
            self.disconnect()
            return False
        
    def on_error(self, status):
        print(status)

# Function to remove all rules from an existing stream_client
def remove_rules(stream_client):
    rules = list(stream_client.get_rules())[0]
    if rules != None:
        for rule in rules:
            id_ = rule.id
            stream_client.delete_rules(id_)
        
s_c = MyStreamingClient(bearer_token=BEARER)
remove_rules(s_c)

In [69]:
s_c.add_rules(tweepy.StreamRule('"Musk"'))
s_c.add_rules(tweepy.StreamRule('"Elon"'))

s_c.get_rules()

Response(data=[StreamRule(value='"Musk"', tag=None, id='1594758661976440832'), StreamRule(value='"Elon"', tag=None, id='1594758663717081088')], includes={}, errors=[], meta={'sent': '2022-11-21T18:24:29.473Z', 'result_count': 2})

In [ ]:
import datetime as dt
print(dt.datetime.now())
s_c.filter(tweet_fields="entities")

2022-11-21 13:25:46.705833


https://shuzhanfan.github.io/2018/03/twitter-streaming-collection/